In [ ]:
!pip install diffusers --upgrade

In [ ]:
!pip install invisible_watermark transformers accelerate safetensors

In [ ]:
!pip install moviepy

In [ ]:
import os
import requests
import json
from huggingface_hub import login
from diffusers import DiffusionPipeline
import torch
import time
import random

In [5]:
from moviepy.editor import VideoFileClip

In [8]:
from diffusers import DiffusionPipeline, DPMSolverMultistepScheduler
from diffusers.utils import export_to_video

In [ ]:

# Generate a GIF from the article title
pipe = DiffusionPipeline.from_pretrained("damo-vilab/text-to-video-ms-1.7b", torch_dtype=torch.float16, variant="fp16")
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
pipe.enable_model_cpu_offload()

def get_news():
    url = f"https://newsapi.org/v2/top-headlines?country=us&apiKey=api_key"
    response = requests.get(url)
    data = response.json()
    articles = data["articles"]
    
    # Select a random article
    article = random.choice(articles)

    # Get the article title and remove the newspaper name
    prompt = article["title"].rsplit(" - ", 1)[0]

    # Get the URL of the article
    article_url = article["url"]
    video_frames = pipe(prompt, num_inference_steps=25).frames
    video_path = export_to_video(video_frames)
    clip = (VideoFileClip(video_path))
    clip_resized = clip.resize(height=800)
    clip_resized.write_gif('output.gif')


    # Update the README file
    version = int(time.time())  # Use the current timestamp as the version
    with open('README.md', 'w') as f:
        f.write("# News GIF Generation\n")
        f.write("This repository contains a Python script that automatically generates a GIF based on a news headline. The script uses the [NewsAPI](https://newsapi.org/) to fetch the latest headlines, and then generates a GIF using the [DiffusionPipeline](https://github.com/Stability-AI) from the `diffusers` Python package.\n")
        f.write("The script is scheduled to run every few minutes. Each time it runs, it fetches a new headline and generates a new GIF, which is then displayed below. The GIF generation process is performed on a GPU.\n")
        f.write("\n## Generated GIF\n")
        f.write("Below is the latest generated GIF:\n")
        f.write(f"![Generated GIF](output.gif?raw=true&v={int(time.time())})\n")
        f.write("\n## Latest News Headline\n")
        f.write("The latest news headline used to generate the GIF is:\n")
        f.write(f"\n**{prompt}**\n")
        f.write(f"\nYou can read more about it [here]({article_url}).\n")
        f.write("\n## Requirements\n")
        f.write("- Python 3.8\n")
        f.write("- Hugging Face account\n")
        f.write("- NewsAPI key\n")
        f.write("- `diffusers` Python package\n")
        f.write("- `invisible_watermark` Python package\n")
        f.write("- `transformers` Python package\n")
        f.write("- `accelerate` Python package\n")
        f.write("- `safetensors` Python package\n")
        f.write("- `moviepy` Python package\n")

get_news()